<a href="https://colab.research.google.com/github/penny1xu/RESTS/blob/main/python_rests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# data clean 

BathroomsFull
BathroomsHalf
BathroomsTotalDecimal
BathroomsTotalInteger
BedroomsTotal
BuildingAreaTotal
City
CloseDate
ClosePrice
CumulativeDaysOnMarket
DaysOnMarket
GarageSpaces
Latitude
ListingContractDate
ListingId
ListPrice
ListSource
Longitude
LotSizeAcres
MlsStatus
OwnerName
OwnerPhone
OwnerPhoneAlternative
ParcelNumber
ParcelNumber2
PoolYN
PostalCode
PreviousListPrice
PreviousStatus
PropertySubType
PropertyType
PublicRemarks
PurchaseContractDate
SchoolDistrict
StandardStatus
StatusChangeTimestamp
StreetDirPrefix
StreetDirSuffix
StreetName
StreetNumber
StreetNumberNumeric
StreetSuffix
SubdivisionName
USProperty_MUI
YearBuilt



In [ ]:
#import package
!pip install censusgeocode
!pip install rets
import csv
import pandas as pd
import censusgeocode as cg
import numpy as np
import sys


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Data cleaning

In [20]:
import pandas as pd
import censusgeocode as cg
import numpy as np
from rets import Session

# select the data columns we want 

def findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, house_leftSqt, house_rightSqt, research, column, zip):
    basic = house_leftSqt
    count = 0
    df = pd.DataFrame()
    result = research.search(resource='Property', resource_class='Property', dmql_query='(MlsStatus = SLD), (CloseDate = 2022-01-01-2022-05-31), (Longitude = {}+), (Longitude = {}-), (Latitude = {}+), (Latitude = {}-), (LivingArea = {}+), (LivingArea = {}+), (PostalCode = {})'.format(house_leftLo, house_rightLo, house_leftLa, house_rightLa, house_leftSqt, house_rightSqt, zip))
    for item in result:
        if(item['PropertyType'] == 'Residential' and item['PropertySubType'] == 'Single Family Residence'):
            df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True)
            count += 1
    if(count >= 5):
        df = df.T
        df.columns = column
        return df
    else:
        house_leftLa -= 0.01
        house_rightLa += 0.01
        house_leftLo -= 0.005
        house_rightLo += 0.005
#先加经纬度，不用限制房子范围
        if(basic <= 1100):
            house_leftSqt -= 100
            house_rightSqt += 100
        elif(basic <= 2200):
            house_leftSqt -= 200
            house_rightSqt += 200 
        else:
            house_leftSqt -= 300
            house_rightSqt += 300
#年份 
        return findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, house_leftSqt, house_rightSqt, research, column, zip) 

def findifNull(lo_left, lo_right, la_left, la_right, research, zip):
    result = research.search(resource='Property', resource_class='Property', dmql_query='(MlsStatus = SLD), (CloseDate = 2022-01-01-2022-05-31), (Longitude = {}+), (Longitude = {}-), (Latitude = {}+), (Latitude = {}-), (PostalCode = {})'.format(lo_left, lo_right, la_left, la_right, zip))
    count = 0
    lis = []
    di = {}
    df = pd.DataFrame()
    for item in result:
        if(item['PropertyType'] == 'Residential' and item['PropertySubType'] == 'Single Family Residence'):
            df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True)
            count += 1
            di = item
    for key in di:
        lis.append(key)
    if(count >= 5):
        df = df.T
        df.columns = lis
        return df
    else:
        lo_left -= 0.001
        lo_right += 0.001
        la_left -= 0.001
        la_right += 0.001
        return findifNull(lo_left, lo_right, la_left, la_right, research, zip)

login_url = 'https://ntrdd.mlsmatrix.com/rets/Login.ashx' 
username = '0671181_NID'
password = 'Rt$tg6jx'
rets_client = Session(login_url, username, password)
rets_client.login()
def findfirst(result):
    count = 0
    for item in result:
        count += 1
    if(count == 0):
        return False
    else:
        return True

def main():
    print('Enter #MLS')
    mls = input()
    #14730621
    pulled_value = {}
    result = rets_client.search(resource='Property', resource_class='Property', dmql_query='(ListingId = {})'.format(mls))
    for item in result:
        pulled_value = item
    if(len(pulled_value) != 0):
        column = []
        for key in pulled_value:
            column.append(key)
        print("\nThis value exists in Website")       
  # get whole row value
  #get each value
        zip = pulled_value.get('PostalCode')
        house_leftLa = float(pulled_value.get('Latitude'))
        house_rightLa = float(pulled_value.get('Latitude')) 
        house_leftLo = float(pulled_value.get('Longitude'))
        house_rightLo = float(pulled_value.get('Longitude'))
        house_leftSqt = float(pulled_value.get('LivingArea'))
        house_rightSqt = float(pulled_value.get('LivingArea'))
        result = findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, house_leftSqt, house_rightSqt, rets_client, column, zip)
        result = result[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt']]
        price = []
        for item in result['ClosePrice']:
            item = float(item)
            price.append(item)
        result['ClosePrice'] = price
        mean = result['ClosePrice'].mean()
        top = list(result['ClosePrice'])
        top.sort(reverse = True)
        return  result.to_csv('result.csv'), mean, top[0:3]
    #.to_csv('result.csv'), mean, top[0:3]
    else:
        print('invalid ID, please insert specific value')
        print('Enter address')
        address = input()
        #'8615 Barclay Street'
        print('Enter city')
        city = input()
        #'Dallas'
        print('Enter state')
        state = input()
        #'Texas'
        print('Enter zipcode')
        zip = input()
        #'75227'
        result = cg.address(address, city = city, state= state,zipcode = zip)
        lo_left = result[0]['coordinates']['x']
        lo_right = result[0]['coordinates']['x']
        la_left = result[0]['coordinates']['y']
        la_right = result[0]['coordinates']['y']
        result = findifNull(lo_left, lo_right, la_left, la_right, rets_client, zip)
        result = result[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt']]
        price = []
        for item in result['ClosePrice']:
            item = float(item)
            price.append(item)
        result['ClosePrice'] = price
        mean = result['ClosePrice'].mean()
        top = list(result['ClosePrice'])
        top.sort(reverse = True)
        return result.to_csv('result.csv'), mean, top[0:3]
  # # limit zipcode range
    
if __name__ == "__main__":
    print(main())

Enter #MLS
14725054

This value exists in Website


KeyboardInterrupt: ignored